<a href="https://colab.research.google.com/github/brianshin12/19-lab/blob/master/RL_Ch2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2장 강화학습 개념

### 2.1 강화학습 개요
- 강화학습 : 원하는 목표를 달성하기 위해 시간 순서대로 시스템에 가해지는 행동(action)을 선택하기 위한 방법.

  - 구성 요소 : 에이전트, 환경, 에이전트와 환경과의 상호작용 행동, 보상, 상태의 관측
  - 시간 변수는 불연속적이며 모든 시간스텝마다 행동이 가해진다.
  - 의사 결정자 : 에이전트(agent)
  - 시스템 : 에이전트의 환경(environment)
  - agent는 환경의 변화를 표현하는 상태(state)을 관측(dbservation)하여 일정한 정책(policy)하에 불연속적인 값이나 연속적인 값으로 된 행동을 선택하여, 이를 환경에 인가해 환경을 변화시기킨다. 그 결과 시간스텝마다 의사 결정 성과를 평가하는 보상(reward)를 제공받는다.
  - 강화학습의 목표 : **종료되는 시점까지 누적된 총 보상을 최대화하는 것.**
  - 순차적인 의사 결정의 특징

    : 일정 시간스텝에서 일시적으로 보상이 작더라도 누적 보상을 최대로 할 수 있는 행동을 선택.
   
- 예시 1 : 그리드 월드

![대체 텍스트](https://github.com/syeong1218/RL/blob/master/2-1.jpg?raw=true)

목표 : 로봇이 (4,3)으로 스스로 찾아가게 해서 최대 보상을 받는 것

에이전트 : 이동방향을 결정하는 로본의 컴퓨터

환경 : 에이전트를 제외한 로봇과 그리드 월드 전체

상태 : 그리드 월드 내에서 로봇의 위치

행동 : 상(U), 하(D), 좌(L), 우(R)

=> 로봇의 위치를 바꿔가며 받는 보상을 통해 시행착오를 겪으면서 R-R-U-U-R,  R-R-R-L-U-U-L-L-R-R-R과 같은 행동을 선택한다. 로봇이 이동할 때마다 보상 -0.01을 받는다고 하면  R-R-U-U-R이나 U-U-R-R-R 중 하나를 선택 할 것이다.

- 최적 제어와 강호학습의 차이점

 : **제어 대상 시스템의 수학적 동역학 모델의 유무**
    - 최적제어 : 수학적 모델을 가지고있어 시스템이 어떻게 동작할지 예측 가능하다.
    - 강화학습 : 수학적 모델을 요구하지 않고 시스템에서 얻은 데이터만 사용한다.

    => 다양한 변수들로 인해 시스템의 모델을 전혀 얻을 수 없는 경우가 있는데, 이 경우에는 강화학습이 좋다.

    - 예시 : 드론이 장애물을 피해가면서 최소의 에너지를 사용해 목표 지점에 도달하도록 드론에 가해지는 제어 입력 또는 행동을 순차적으로 결정하는 것이 목표.

![대체 텍스트](https://github.com/syeong1218/RL/blob/master/2-3.jpg?raw=true)

| |최적제어|강화학습|
|----|----|----|
|비행 제어 컴퓨터|제어기|에이전트|
|컴퓨터를 제외한 모든 것|시스템|환경|
||수학적 모델에 의존|보상에 의존|

![대체 텍스트](https://github.com/syeong1218/RL/blob/master/2-4.jpg?raw=true)
=> 최적 제어는 비행 제어 컴퓨터를 제외한 모든 환경에 대한 수학적 모델을 알고 있고, 강화학습은 아무것도 모른다.


### 2.2 강화학습 프로세스와 표기법

![대체 텍스트](https://github.com/syeong1218/RL/blob/master/2-5.jpg?raw=true)

(1) 에이전트는 환경의 상태($x_t$)를 측정한다.

(2) 측정한 상태에서 에이전트의 정책으로 선택한 행동($u_t$)을 환경에 보낸다.

    정책 : 측정한 상태를 바탕으로 최선의 행동을 선택하기 위한 에이전트의 규칙 또는 방법

(3) 행동에 의해서 환경의 상태는 다음 상태($x_{t+1}$)로 전환된다.

(4) 전환된 환경의 상태를 바탕으로 다시 에이전트는 새로운 행동을 선택한다.

 (5) 환경으로부터 주어지는 즉각적인 보상($r_{t+1}$)을 사용해 장기적인 성과를 계산 또는 예측해서 에이전트의 정책을 개선한다.

 => 이와 같이 시간스텝마다 에이전트가 새로운 보상을 받는 것을 반복하는 과정을 수학적으로 모델링한 것이 마르코프 결정 프로세스(MDP)이다.

# 3장 정책 그래디언트  

## 3.1 배경  
강화학습 문제를 해결하는 모든 방법의 최종 목표는 노적 보상을 최대화하는 최적 정책(Optimal policy)을 구하는 것이다.  
이 장에서는 정책(policy)를 파라미터화하고, 누적 보상을 파라미터화된 정책의 함수로 기술하여 누적 보상과 정책 파라미터 간의 관계를 구축하고 최적화 방법을 통해 누적 보상을 최대화하는 정책 파라미터를 계산하는 강화학습의 방법론 중 정책 그래디언트를 알아볼 것이다.

## 3.2 목적함수  
강화학습의 목표를 수학적으로 표현하면 반환값의 기댓값으로 이루어진 목적함수 __J__를 최대로 만드는 정책 $\pi(u_t|x_t)$을 구하는 것이다.  
정책이 $\theta$로 파라미터화 됐다면 $\pi_\theta(u_t|x_t)$로 표현하고 목적함수를 최대로 만드는 정책 파라미터 $\theta$를 계한하는 것이라 할 수 있다.  
$$\theta^*=argmaxJ(\theta)$$
$$J(\theta)=E_{\tau \sim p_\theta(\tau)}[\sum_{t=0}^T \gamma^t r(x_t,u_t)] $$  
여기서 $r(x_t, u_t)$는 시간스텝 $t$일 때 상태변수가 $x_t$에서 행동 $u_t$를 사용했을 때 에이전트가 받는 보상을 나타내며, $\gamma\in[0,1]$은 감가율(discount factor)이다. 기댓값의 아래 첨자 $p_\theta(\tau)$는 기댓값을 계산할 때 확률밀도함수로 $p_\theta(\tau)$를 사용한다는 의미다.  
그림 3.1과 같이 $\tau$는 정책 $\pi_\theta$로 생성되는 궤적 $\tau=(x_0, u_0, x_1, u_1, x_2, u_2, x_3, u_3, \cdots, x_T, u_T)$이다. $p_\theta(\tau)$는 정책 $\pi_\theta$로 생성되는 궤적의 확률밀도함수를 나타낸다.

![KakaoTalk_20200509_145820465](https://user-images.githubusercontent.com/53211502/81465554-c4a71f00-9205-11ea-852f-4aa6c861bb7b.jpg)  

정책은 보통 신경망(neural network)으로 파라미터화된다. 이 신겸망을 정책 신경망(policy neural network)이라고 하며, 파라미터 $\theta$는 그림 3.2에 도시한 것 같이 신경망의 모든 가중치다.  
![KakaoTalk_20200509_151311299](https://user-images.githubusercontent.com/53211502/81465828-f0c39f80-9207-11ea-968a-db245cc2df55.jpg)

에이전트의 정책 신경망과 환경과의 상호작용을 마르코프 결정 프로세스(MDP) 프레임워크로 표현하면 다음 그림 3.3과 같다.
![KakaoTalk_20200509_152306979](https://user-images.githubusercontent.com/53211502/81465989-0e453900-9209-11ea-96c8-92d6c716625e.jpg)

$G_0 = \sum_{t=0} ^T \gamma^t r (x_t, u_t)$는 시간 스텝 $t=0$부터 에피소드가 종료될 때까지 받을 수 있는 전체 궤적에 대한 감가 보상(discount reward)의 총합으로서, 전체 반환값 $G_0$라고 부른다. 임의의 시간 $k=t$부터 에피소드가 종료될 때까지 받을 수 있는 예정 보상(reward-to-go)의 총합은 $G_k = \sum_{k=t}^T \gamma^{k-t} r (x_k, u_k)$로 표시한다.


$p_\theta (\tau) = p_\theta(x_0, u_0, x_1, u_1, \cdots. x_T, u_T)$  
전개된 $p_\theta (\tau)$를 베이즈 정리와 마르코프 시퀀스 가정에 의해 다음과 같이 표현할 수 있다.  
$$p_\theta (\tau) = p(x_0) \prod_{t=0}^T \pi_\theta(u_t|x_t)p(x_{t+1}|x_t, u_t)$$  

*  베이즈 정리  
$$P\{A\} = \sum_{i=1}^n P\{A, B_i\} = \sum_{i=1}^n P\{A|B_i\} P\{B_i\}$$  
* 마르코프 시퀀스 가정  
 현재의 state가 history의 모든 관련 정보를 갖고 있다.  

목적함수 $J$를 전개하면 다음과 같다.  
$$J(\theta)=E_{\tau \sim p_\theta(\tau)}[\sum_{t=0}^T \gamma^t r(x_t,u_t)] 
           =\int_\tau p_\theta(\tau)(\sum_{t=0} ^T \gamma^t r (x_t, u_t))d\tau $$  
여기서 궤적을 $x_0$과 나머지, 두 영역으로 분할하고 베이즈 정리를 적용하면 다음과 같다.  
$$p_\theta (\tau) = p(x_0) p_\theta(\tau_{u_0 : u_T}|x_0)$$
위 식을 목적함수 전개 식에 대입 후 상태가치 함수 $V^{\pi_\theta}$로 묶어 주면 목적함수는 다음과 같다.  
$$J(\theta) = E_{x_0 \sim p(x_0)}[V^{\pi_\theta}(x_0)]$$  
즉 목적함수는 초기 상태변수 $x_0$에 대한 상태가치의 평균값이 된다.

## 3.3 정책 그래디언트  
목적함수를 최대로 만드는 $\theta$를 계산하기 위해 목적함수를 $\theta$로 미분해 보자.  
$$ {{\partial J(\theta)}\over{\partial \theta}}= \nabla_\theta J(\theta) = \int_\tau \nabla_\theta p_\theta(\tau) \sum_{t=0}^T \gamma^t r (x_t, u_t) d\tau=\int_\tau p_\theta(\tau) \nabla_\theta \log p_\theta(\tau) \sum_{t=0}^T \gamma^t r (x_t, u_t) d\tau$$  위 식을 목적함수의 그래디언트 식이라 한다.
$\nabla_\theta \log p_\theta(\tau) = {{\nabla_\theta p_\theta(\tau)}\over{p_\theta(\tau)}}$를 이용하여 log를 전개하면 다음과 같다.  
$$\nabla_\theta \log p_\theta(\tau) = \nabla_\theta (\log p(x_0) + \sum_{t=0}^T \log \pi_\theta (u_t|x_t) + \log p(x_{t+1}|x_t, u_t))$$  
좌변의 두번째 항만 $\theta$의 함수이므로 $\nabla_\theta \log p_\theta(\tau) =  \sum_{t=0}^T \nabla_\theta \log \pi_\theta (u_t|x_t)$이다. 이 식을 목적함수의 그래디언트 식에 대입하면 환경 모델이 필요 없는 모델 프리(model-free) 강화학습 방법이 되고 그 식은 다음과 같다.  
$$\nabla_\theta J(\theta)=E_{\tau \sim p_\theta(\tau)}[\sum_{t=0}^T (\nabla_\theta \log \pi_\theta (u_t|x_t)(\sum_{k=0}^T\gamma^k r (x_k, u_k)))] $$  
위 식에 시간스텝 t<k에서의 보상값에 영향을 끼치지 못한다는 점(인과성)을 고려하고 무한 구간 에피소드에서도 적용 가능하게 하기 위해 예정 보상에만 감가율을 적용하면 다음과 같다.  
$$\nabla_\theta J(\theta)=E_{\tau \sim p_\theta(\tau)}[\sum_{t=0}^T (\nabla_\theta \log \pi_\theta (u_t|x_t)(\sum_{k=t}^T\gamma^{k-t} r (x_k, u_k)))] $$  
목적함수를 최대로 하는 파라미터 $\theta$는 다음과 같이 경사상승법으로 구할 수 있다.  
$$\theta \leftarrow \theta + \alpha \nabla_\theta J(\theta)$$  

정책 그래디언트에 사용되는 목적함수 그래디언트를 다음과 같다.  
* 목적함수 : $J(\theta)=E_{\tau \sim p_\theta(\tau)}[\sum_{t=0}^T \gamma^t r(x_t,u_t)], \tau=(x_0, u_0. x_1. u_1, \cdots, x_T, u_T)$  
* 가정 : 확률적 정책, $u_t \sim \pi_\theta (u_t|x_t)$  
* 그래디언트 : $\nabla_\theta J(\theta)=E_{\tau \sim p_\theta(\tau)}[\sum_{t=0}^T (\nabla_\theta \log \pi_\theta (u_t|x_t)(\sum_{k=t}^T\gamma^{k-t} r (x_k, u_k)))] $
* 업데이트 : $\theta \leftarrow \theta + \alpha \nabla_\theta J(\theta)$  


## 3.4 REINFORCE 알고리즘  
정책 그래디언트를 실제 적용하는 데 있어서 $\tau$상의 기댓값은 에피소드를 M개만큼 샘플링해 에피소드 평균을 이요해서 근사적으로 계산한다. 위 방법을 이용하면 목적함수의 그래디언트는 다음과 같이 근사적으로 추정할 수 있다.  
 $$\nabla_\theta J(\theta) \approx \nabla_\theta {1 \over M} \sum_{m=1}^M[\sum_{t=0}^T ( \log \pi_\theta (u_t^{(m)}|x_t^{(m)})G_t^{(m)})]$$  

 M개의 에피소드를 이용해 목적함수의 그래디언트를 계산하는 식을 도시하면 다음과 같다.  
 ![KakaoTalk_20200509_171445690](https://user-images.githubusercontent.com/53211502/81468127-a4cd2680-9218-11ea-9432-fa672b5d0661.jpg)

한 개의 에피소드를 이용해 목적함수의 그래디언트를 계산하는 방법을 도시하면 다음과 같다.  
![KakaoTalk_20200509_171808860](https://user-images.githubusercontent.com/53211502/81468209-202ed800-9219-11ea-8bc1-8fb3fdaa080b.jpg)

파라미터 $\theta$로 표현된 정책 $\pi_\theta$을 신경망으로 구성할 때  
$\theta \leftarrow \theta + \alpha \nabla_\theta J(\theta) \approx \alpha \nabla_\theta \sum_{t=0}^T ( \log \pi_\theta (u_t^{(m)}|x_t^{(m)})G_t^{(m)})$이기 때문에  
에피소드의 손실함수로 $loss = -\sum_{t=0}^T ( \log \pi_\theta (u_t^{(m)}|x_t^{(m)})G_t^{(m)})$를 사용한다.  
따라서 반환값을 크게 받은 정책의 에피소드는 목적함수의 그래디언트 계산 시 더 큰 영향을 끼치고 반환값이 작은 정책의 에피소드는 작은 영향을 끼쳐, 점진적으로 정책이 개선된다.  

다음은 REINFORCE 알고리즘의 프로세스이다.  
![KakaoTalk_20200509_173230345](https://user-images.githubusercontent.com/53211502/81468581-6be28100-921b-11ea-97c4-572781ac3eae.jpg)

정책 $\pi_{\theta_1}$을 에피소드가 종료할 때까지 실행시켜 $(T_1+1)$개의 샘플 $(x_t, u_t, r(x_t, u_t), x_{t+1})_{t=0, \dots, T_1}$을 생성하고, 이를 바탕으로 정책을 $\pi_{\theta_1}$에서 $\pi_{\theta_2}$로 업데이트한다. 그 후 $(T_1+1)$개의 샘플은 바로 폐기하고, 다시 업데이트된 정책 $\pi_{\theta_2}$를 다른 에피소드가 종료할 때까지 실행시켜 다시 $(T_2+1)$개의 새로운 샘플$(x_t, u_t, r(x_t, u_t), x_{t+1})_{t=0, \dots, T_2}$를 생성한다. 그리고 정책$\pi_{\theta_2}$를 새로운 정책으로 업데이트한다. 이러한 과정을 일정 학습 성과에 도달할 때까지 반복하면서 학습을 진행한다.  

REINFORCE 알고리즘에는 몇 가지 문제가 있다.  
1. 한 에피소드가 끝나야 정책을 업데이트할 수 있다.  
$\rightarrow$ 신경망 학습 시간이 상당히 길어질 수 있다.  
2. 그래디언트의 분산이 매우 크다.  
$\rightarrow$ 반환값이 에피소드마다 크게 차이가 남에 따라 목적함수의 그래디언트도 이에 비례하여 값이 들쭉날쭉하므로 신경망 학습에 시간이 많이 걸리고 학습이 전혀 안 될 수도 있다.  
3. 온-플리시(on-policy) 방법이다.  
$\rightarrow$ 정책을 업데이트하기 위해서 해당 정책을 실행시켜 발생한 에피소드가 필요하므로 효율성이 매우 떨어진다.  

하지만 REINFORCE 알고리즘은 다른 정책 그래디언트 기반 알고리즘의 기초가 된다는 점에서 그 의의가 있다.